In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import *
import numpy as np
import matplotlib.pyplot as plt

In [4]:
quizzes = np.zeros((1000000, 81), np.int32)
solutions = np.zeros((1000000, 81), np.int32)
for i, line in enumerate(open('sudoku.csv', 'r').read().splitlines()[1:]):
    quiz, solution = line.split(",")
    for j, q_s in enumerate(zip(quiz, solution)):
        q, s = q_s
        quizzes[i, j] = q
        solutions[i, j] = s
X = quizzes.reshape((-1, 9, 9))
Y = solutions.reshape((-1, 9, 9))

In [23]:
# convert X and Y to tf tensors. Allows GPU accelleration
X = tf.convert_to_tensor(X, dtype=None, dtype_hint=None, name=None)
Y = tf.convert_to_tensor(Y, dtype=None, dtype_hint=None, name=None)
# splits X and Y into train dev and test splits
Xtr, X_t = tf.split(X, num_or_size_splits=[800000, 200000])
Ytr, Y_t = tf.split(Y, num_or_size_splits=[800000, 200000])
Xdev, Xte = tf.split(X_t, num_or_size_splits=2)
Ydev, Yte = tf.split(Y_t, num_or_size_splits=2)

In [32]:
Xtr[0].shape

TensorShape([9, 9])

In [38]:
model = keras.models.Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(9, 9, 1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 7, 7, 64)          640       
                                                                 
 batch_normalization_2 (Batc  (None, 7, 7, 64)         256       
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 5, 5, 64)          36928     
                                                                 
 flatten_2 (Flatten)         (None, 1600)              0         
                                                                 
 dense_2 (Dense)             (None, 729)               1167129   
                                                                 
 reshape_2 (Reshape)         (None, 81, 9)             0         
                                                      

In [40]:
model.fit(Xtr, Ytr, batch_size=64, epochs=5)

Epoch 1/5


TypeError: 'NoneType' object is not callable